# About: Dockerのインストール

Ubuntu 14.04環境にDockerをインストールするためのNotebook。

このNotebookによりインストールできるソフトウェアは以下の通り。

- Docker Engine
- Docker Compose

Docker Engineのインストール手順は、[Installation on Ubuntu](https://docs.docker.com/engine/installation/ubuntulinux/) (*2016/6/3時点での最新版*) を参考に作成している。また、Docker Composeのインストール手順は、[Install Docker Compose](https://docs.docker.com/compose/install/) (*2016/6/3時点での最新版*) を参考に作成している。

## Operation Note

*お手本の作成用*


## 設定情報

このNotebookで行う設定は、以下のようにする。

- NIIのベアメタルマシンを想定 ... Docker関係のディレクトリは `/mnt`　(Ephemeralなパーティション) に配置する
- クラウド運用チームでの利用を想定 ... Dockerのプライベートレジストリを使用するので、プライベートレジストリのホスト情報を明示する

する。

docker_optsの定義方法は[Command line reference / daemon](https://docs.docker.com/engine/reference/commandline/dockerd/)を参照。

In [1]:
docker_tmp = "/mnt/docker-tmp"
docker_base = "/mnt/docker"
docker_opts ="-g {docker_base} --insecure-registry XXX.XXX.XXX.93:5000".format(docker_base=docker_base)

# Notebookと環境のBinding

Inventory中のgroup名でBind対象ホスト(Docker Engineをインストールしたいホスト)を指示する。

In [3]:
target_group = 'test-vm'

Bind対象への疎通状態を確認する。

In [4]:
!ansible -m ping {target_group}

XXX.XXX.XXX.66 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


# Binding対象の確認

[Prerequisites](https://docs.docker.com/engine/installation/linux/ubuntulinux/#prerequisites)に示されているとおり、このNotebookを使ってDockerをインストールする対象のホストは、以下の条件を満たしている必要がある。

もし、このインストール手順に失敗したら、**この条件を満たす状態にマシンを戻す(パッケージの削除, マシンの再プロビジョニングなど)**ことで、**(このNotebookによって)Dockerをインストール可能な状態に戻す**ことができる。

## 64bit版を使う

Ubuntuのバージョンにかかわらず、Docker Engineを動作させるには64bit版が必要。

In [5]:
!ansible -a 'uname -m' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
x86_64



## kernel versionは最低3.10

3.10未満の古いバージョンの場合はDockerの機能の一部が使えなかったり、データロストやpanicを生じる可能性がある。

そのため、以下のバージョン表示が3.10以上であることを確認しておく。

In [6]:
!ansible -a 'uname -r' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
3.13.0-88-generic



## Ubuntuは14.04を想定

加えて、このNotebookは、**Ubuntu 14.04がインストールされた環境にBindingされることを前提として実装**している。

以下のコマンドの出力が Ubuntu 14.04 であることを確認する。

> 14.04以外のUbuntuの場合はRepositoryのURLなどを適宜読み替えること

In [7]:
!ansible -a 'lsb_release -a' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
Distributor ID:	Ubuntu
Description:	Ubuntu 14.04.4 LTS
Release:	14.04
Codename:	trustyNo LSB modules are available.



## Ubuntu-maintainedなDockerがインストールされていないこと

Ubuntu-maintainedなDocker(`docker-io` パッケージ)がすでにインストールされているとファイル構成など競合するかもしれない。そのため、念のため以下のコマンドに**失敗する(FAILED)** ことを確認しておく。

In [8]:
!ansible -b -m shell -a 'dpkg -l | grep docker.io' {target_group}

XXX.XXX.XXX.66 | FAILED | rc=1 >>




## 古いrepositoryからインストールされていないこと

古いrepositoryでは `lxc-docker` というパッケージ名だった時代があった・・・これがインストールされていないことも念のため確認しておく。以下のコマンドに**失敗する(FAILED)** ことを確認しておく。

In [9]:
!ansible -b -m shell -a 'dpkg -l | grep lxc-docker' {target_group}

XXX.XXX.XXX.66 | FAILED | rc=1 >>




## APTにdocker-engineのrepositoryが未登録

このNotebook適用時は、docker-engineのrepositoryが未登録であることを前提としている。すでにrepositoryが登録されている場合、このNotebookで指定したパッケージが適切にインストールされないかもしれない。念のため以下のコマンドの実行結果に、**何もインストール候補バージョンが現れない**ことを確認しておく。

In [10]:
!ansible -b -m shell -a 'apt-get update && apt-cache policy docker-engine' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
Ign http://us.archive.ubuntu.com trusty InRelease
Get:1 http://security.ubuntu.com trusty-security InRelease [65.9 kB]
Get:2 http://us.archive.ubuntu.com trusty-updates InRelease [65.9 kB]
Hit http://us.archive.ubuntu.com trusty-backports InRelease
Get:3 http://security.ubuntu.com trusty-security/main Sources [117 kB]
Hit http://us.archive.ubuntu.com trusty Release.gpg
Get:4 http://us.archive.ubuntu.com trusty-updates/main Sources [277 kB]
Get:5 http://security.ubuntu.com trusty-security/restricted Sources [4,035 B]
Get:6 http://security.ubuntu.com trusty-security/universe Sources [37.3 kB]
Get:7 http://us.archive.ubuntu.com trusty-updates/restricted Sources [5,352 B]
Get:8 http://security.ubuntu.com trusty-security/multiverse Sources [2,757 B]
Get:9 http://us.archive.ubuntu.com trusty-updates/universe Sources [156 kB]
Get:10 http://security.ubuntu.com trusty-security/main amd64 Packages [493 kB]
Get:11 http://us.archive.ubuntu.com trusty-updates/mult

# Docker Engineのインストール

Bind対象にDocker Engineをインストールする。

## apt sourcesの更新

Docker社のrepositoryの情報をBind対象マシンに追加する。

まず、HTTPSのrepositoryからパッケージをインストールできるようにしておく。

In [11]:
!ansible -b -m shell -a 'apt-get update && \
                         apt-get install -y apt-transport-https ca-certificates' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
Hit http://security.ubuntu.com trusty-security InRelease
Ign http://us.archive.ubuntu.com trusty InRelease
Hit http://us.archive.ubuntu.com trusty-updates InRelease
Hit http://security.ubuntu.com trusty-security/main Sources
Hit http://us.archive.ubuntu.com trusty-backports InRelease
Hit http://security.ubuntu.com trusty-security/restricted Sources
Hit http://us.archive.ubuntu.com trusty Release.gpg
Hit http://security.ubuntu.com trusty-security/universe Sources
Hit http://us.archive.ubuntu.com trusty-updates/main Sources
Hit http://security.ubuntu.com trusty-security/multiverse Sources
Hit http://us.archive.ubuntu.com trusty-updates/restricted Sources
Hit http://security.ubuntu.com trusty-security/main amd64 Packages
Hit http://us.archive.ubuntu.com trusty-updates/universe Sources
Hit http://security.ubuntu.com trusty-security/restricted amd64 Packages
Hit http://us.archive.ubuntu.com trusty-updates/multiverse Sources
Hit http://security.ubuntu.com t

GPG keyを追加しておく。

In [12]:
!ansible -b -a 'apt-key adv \
                        --keyserver hkp://p80.pool.sks-keyservers.net:80 \
                        --recv-keys 58118E89F3A912897C070ADBF76221572C52609D' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
Executing: gpg --ignore-time-conflict --no-options --no-default-keyring --homedir /tmp/tmp.CLRwFhjYQl --no-auto-check-trustdb --trust-model always --keyring /etc/apt/trusted.gpg --primary-keyring /etc/apt/trusted.gpg --keyserver hkp://p80.pool.sks-keyservers.net:80 --recv-keys 58118E89F3A912897C070ADBF76221572C52609Dgpg: requesting key 2C52609D from hkp server p80.pool.sks-keyservers.net
gpg: key 2C52609D: public key "Docker Release Tool (releasedocker) <docker@docker.com>" imported
gpg: Total number processed: 1
gpg:               imported: 1  (RSA: 1)



Ubuntuのバージョンに応じたrepositoryのURLを追加する。このNotebookでは **14.04(Trusty)** を追加。

In [13]:
!ansible -b -m shell -a 'echo deb https://apt.dockerproject.org/repo ubuntu-trusty main \
                              > /etc/apt/sources.list.d/docker.list' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>




aptがrepositoryから情報を取得できているかの確認。

In [14]:
!ansible -b -m shell -a 'apt-get update && \
                         apt-cache policy docker-engine' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
Ign http://us.archive.ubuntu.com trusty InRelease
Hit http://security.ubuntu.com trusty-security InRelease
Hit http://us.archive.ubuntu.com trusty-updates InRelease
Get:1 https://apt.dockerproject.org ubuntu-trusty InRelease
Get:2 https://apt.dockerproject.org ubuntu-trusty/main amd64 Packages
Get:3 https://apt.dockerproject.org ubuntu-trusty/main i386 Packages
Hit http://us.archive.ubuntu.com trusty-backports InRelease
Hit http://security.ubuntu.com trusty-security/main Sources
Hit http://us.archive.ubuntu.com trusty Release.gpg
Get:4 https://apt.dockerproject.org ubuntu-trusty/main Translation-en_US
Hit http://us.archive.ubuntu.com trusty-updates/main Sources
Hit http://security.ubuntu.com trusty-security/restricted Sources
Hit http://us.archive.ubuntu.com trusty-updates/restricted Sources
Hit http://security.ubuntu.com trusty-security/universe Sources
Hit http://us.archive.ubuntu.com trusty-updates/universe Sources
Hit http://security.ubuntu.com tr

## aufs storage driverを利用可能に

Ubuntu 14.04の場合、`linux-image-extra` パッケージのインストールが推奨されている。

In [15]:
!ansible -b -m shell -a 'apt-get update && \
                         apt-get install -y linux-image-extra-$(uname -r)' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
Hit http://security.ubuntu.com trusty-security InRelease
Ign http://us.archive.ubuntu.com trusty InRelease
Hit https://apt.dockerproject.org ubuntu-trusty InRelease
Hit https://apt.dockerproject.org ubuntu-trusty/main amd64 Packages
Hit https://apt.dockerproject.org ubuntu-trusty/main i386 Packages
Get:1 https://apt.dockerproject.org ubuntu-trusty/main Translation-en_US
Ign https://apt.dockerproject.org ubuntu-trusty/main Translation-en_US
Ign https://apt.dockerproject.org ubuntu-trusty/main Translation-en
Hit http://us.archive.ubuntu.com trusty-updates InRelease
Hit http://security.ubuntu.com trusty-security/main Sources
Hit http://us.archive.ubuntu.com trusty-backports InRelease
Hit http://security.ubuntu.com trusty-security/restricted Sources
Hit http://us.archive.ubuntu.com trusty Release.gpg
Hit http://security.ubuntu.com trusty-security/universe Sources
Hit http://us.archive.ubuntu.com trusty-updates/main Sources
Hit http://security.ubuntu.com t

## apparmorのインストール

apparmorもrequiredとなっているので、インストールしておく。

In [16]:
!ansible -b -a 'apt-get install -y apparmor' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
Reading package lists...
Building dependency tree...
Reading state information...
apparmor is already the newest version.
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.



## パッケージのインストール

`docker-engine` パッケージをインストールする。

In [17]:
!ansible -b -m shell -a 'apt-get update && \
                         apt-get install -y docker-engine' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
Ign http://us.archive.ubuntu.com trusty InRelease
Hit https://apt.dockerproject.org ubuntu-trusty InRelease
Hit http://security.ubuntu.com trusty-security InRelease
Hit https://apt.dockerproject.org ubuntu-trusty/main amd64 Packages
Hit https://apt.dockerproject.org ubuntu-trusty/main i386 Packages
Get:1 https://apt.dockerproject.org ubuntu-trusty/main Translation-en_US
Hit http://us.archive.ubuntu.com trusty-updates InRelease
Ign https://apt.dockerproject.org ubuntu-trusty/main Translation-en_US
Ign https://apt.dockerproject.org ubuntu-trusty/main Translation-en
Hit http://us.archive.ubuntu.com trusty-backports InRelease
Hit http://security.ubuntu.com trusty-security/main Sources
Hit http://us.archive.ubuntu.com trusty Release.gpg
Hit http://security.ubuntu.com trusty-security/restricted Sources
Hit http://us.archive.ubuntu.com trusty-updates/main Sources
Hit http://security.ubuntu.com trusty-security/universe Sources
Hit http://us.archive.ubuntu.com

## Docker Engineの設定変更

あらかじめ定義した設定情報にしたがい、Docker Engineに与えるDefault Configを指定する。

In [18]:
import tempfile
temp_dir = tempfile.mkdtemp()
temp_dir

'/tmp/tmppOH4Qq'

In [19]:
import os
with open(os.path.join(temp_dir, 'docker_config'), 'w') as f:
    f.write('''# Docker Upstart and SysVinit configuration file

export TMPDIR="{docker_tmp}"
DOCKER_OPTS="{docker_opts}"'''.format(docker_tmp=docker_tmp, docker_opts=docker_opts))
!cat {temp_dir}/docker_config

# Docker Upstart and SysVinit configuration file

export TMPDIR="/mnt/docker-tmp"
DOCKER_OPTS="-g /mnt/docker --insecure-registry XXX.XXX.XXX.93:5000"

ローカルに作った configファイル を、Bind対象の/etc/default/dockerにコピーし、Docker Engineに反映する。

In [20]:
!ansible -b -m copy -a 'src={temp_dir}/docker_config dest=/etc/default/docker' {target_group}
!ansible -b -m file -a 'path={docker_tmp} state=directory' {target_group}
!ansible -b -m service -a 'name=docker state=restarted' {target_group}

XXX.XXX.XXX.66 | SUCCESS => {
    "changed": true, 
    "checksum": "4995e08c9844f9944b9449c0c6b771bb9694b7ec", 
    "dest": "/etc/default/docker", 
    "gid": 0, 
    "group": "root", 
    "md5sum": "ddcb1600fc8c55d04e1afc73d58257d9", 
    "mode": "0644", 
    "owner": "root", 
    "size": 148, 
    "src": "/home/ansible/.ansible/tmp/ansible-tmp-1466171605.71-252050334803597/source", 
    "state": "file", 
    "uid": 0
}
XXX.XXX.XXX.66 | SUCCESS => {
    "changed": true, 
    "gid": 0, 
    "group": "root", 
    "mode": "0755", 
    "owner": "root", 
    "path": "/mnt/docker-tmp", 
    "size": 4096, 
    "state": "directory", 
    "uid": 0
}
XXX.XXX.XXX.66 | SUCCESS => {
    "changed": true, 
    "name": "docker", 
    "state": "started"
}


念のため、Docker Engineにより /mnt/docker, /mnt/docker-tmp にファイルが作成されていることを確認する。

In [21]:
!ansible -b -a 'ls -la {docker_tmp} {docker_base}' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
/mnt/docker:
total 36
drwx--x--x 9 root root 4096 Jun 17 22:53 .
drwxr-xr-x 4 root root 4096 Jun 17 22:53 ..
drwx------ 5 root root 4096 Jun 17 22:53 aufs
drwx------ 2 root root 4096 Jun 17 22:53 containers
drwx------ 3 root root 4096 Jun 17 22:53 image
drwxr-x--- 3 root root 4096 Jun 17 22:53 network
drwx------ 2 root root 4096 Jun 17 22:53 tmp
drwx------ 2 root root 4096 Jun 17 22:53 trust
drwx------ 2 root root 4096 Jun 17 22:53 volumes

/mnt/docker-tmp:
total 8
drwxr-xr-x 2 root root 4096 Jun 17 22:53 .
drwxr-xr-x 4 root root 4096 Jun 17 22:53 ..



Docker Engineのバージョンを確認する。

In [22]:
!ansible -b -a 'docker version' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
Client:
 Version:      1.11.2
 API version:  1.23
 Go version:   go1.5.4
 Git commit:   b9f10c9
 Built:        Wed Jun  1 21:47:50 2016
 OS/Arch:      linux/amd64

Server:
 Version:      1.11.2
 API version:  1.23
 Go version:   go1.5.4
 Git commit:   b9f10c9
 Built:        Wed Jun  1 21:47:50 2016
 OS/Arch:      linux/amd64



# Docker Composeのインストール

*2016/6/3時点* では、docker-composeのバージョンは1.7.1となる。

In [23]:
!ansible -b -m shell \
         -a 'curl -L https://github.com/docker/compose/releases/download/1.7.1/docker-compose-`uname -s`-`uname -m` \
                 > /usr/local/bin/docker-compose' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   600    0   600    0     0    683      0 --:--:-- --:--:-- --:--:--   683
100 7776k  100 7776k    0     0   427k      0  0:00:18  0:00:18 --:--:--  620k



In [24]:
!ansible -b -a 'chmod +x /usr/local/bin/docker-compose' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>




In [25]:
!ansible -b -a 'docker-compose --version' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
docker-compose version 1.7.1, build 0a9ab35



# Docker Engineの動作確認

まずはお試しで、hello-worldイメージを実行してみる。`Hello from Docker`のようなメッセージが表示されたらOK。

In [26]:
!ansible -b -a 'docker run hello-world' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>

Hello from Docker.
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker Hub account:
 https://hub.docker.com

For more examples and ideas, visit:
 https://docs.docker.com/engine/userguide/Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world
a9d36faac0fe: Pulling fs layer
a9d36faac0fe: Verifying Checksum
a9d3

Dockerのhello-worldイメージが実行された。OK。

# Docker Composeの動作確認

Docker Composeが動作することも確認しておく。

まずローカルにdocker-compose.ymlファイルを準備。

In [27]:
!mkdir -p {temp_dir}/hello-compose/

In [28]:
%%writefile {temp_dir}/hello-compose/docker-compose.yml
version: '2'
services:
  test-hello-world:
    image: hello-world

Writing /tmp/tmppOH4Qq/hello-compose/docker-compose.yml


作成したdocker-compose.ymlを、Bind対象ホストにアップロードする。

In [29]:
!ansible -b -m copy -a 'src={temp_dir}/hello-compose dest=~' {target_group}

XXX.XXX.XXX.66 | SUCCESS => {
    "changed": true, 
    "checksum": "19f3b6e911e6461b09d1ddaf5a5f7f8dbc538a6e", 
    "dest": "/root/hello-compose/docker-compose.yml", 
    "gid": 0, 
    "group": "root", 
    "md5sum": "a157d75e4107e5c1aa63441b9ff90758", 
    "mode": "0644", 
    "owner": "root", 
    "size": 65, 
    "src": "/home/ansible/.ansible/tmp/ansible-tmp-1466171662.93-184926426460876/source", 
    "state": "file", 
    "uid": 0
}


実行してみる。`Hello from Docker`のようなメッセージが表示されたらOK。

In [30]:
!ansible -b -a 'chdir=~/hello-compose docker-compose up' {target_group}

XXX.XXX.XXX.66 | SUCCESS | rc=0 >>
Attaching to hellocompose_test-hello-world_1
test-hello-world_1  | 
test-hello-world_1  | Hello from Docker.
test-hello-world_1  | This message shows that your installation appears to be working correctly.
test-hello-world_1  | 
test-hello-world_1  | To generate this message, Docker took the following steps:
test-hello-world_1  |  1. The Docker client contacted the Docker daemon.
test-hello-world_1  |  2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
test-hello-world_1  |  3. The Docker daemon created a new container from that image which runs the
test-hello-world_1  |     executable that produces the output you are currently reading.
test-hello-world_1  |  4. The Docker daemon streamed that output to the Docker client, which sent it
test-hello-world_1  |     to your terminal.
test-hello-world_1  | 
test-hello-world_1  | To try something more ambitious, you can run an Ubuntu container with:
test-hello-world_1  |  $ docker run -

# 後始末

一時ディレクトリを削除する。

In [31]:
!rm -fr {temp_dir}